In [10]:
from scripts.processing import predict, load_audio
import onnxruntime as ort
import matplotlib.pyplot as plt
import numpy as np

In [5]:
model_type="YOLO" #select model YOLO / CRNN
providers=['CPUExecutionProvider']

if model_type=="YOLO":
  ort_sess = ort.InferenceSession('models/YOLOv5n.onnx', providers=providers)
elif model_type=="CRNN":
  ort_sess = ort.InferenceSession('models/CRNN.onnx',providers=providers)

In [11]:
th=0.3 #Threshold of the confidence of the prediction
Audio_dir = "samples/0.wav" #Audio to predict
dir_audio=Audio_dir
sr=16000
audio=load_audio(dir_audio,sr)
classes=["Horn","Siren","Pets","Physiological","Speech","RingTone","Vibrating","Notifications","Cry"] #List of predictions
p,_=predict(audio, ort_sess,model_type,th) #Do the prediction
print(p)

[array([1.61659801, 4.22585249, 0.        , 0.93429101]), array([8.28870296, 9.18547153, 2.        , 0.89694166])]
